Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Develop Scoring Script

In this notebook, we will develop the scoring script and test it locally. We will use the scoring script to create the 
web service that will call the model for scoring.

In [ ]:
import sys
import pandas as pd

from azure_utils.machine_learning.utils import get_workspace_from_config
from azure_utils.utilities import text_to_json
from azureml.core.model import Model

In [ ]:
sys.path.append('./scripts/')

Let's load the workspace.

In [ ]:
ws = get_workspace_from_config()
print(ws.name, ws.resource_group, ws.location, sep="\n")

Let's retrieve the model registered earlier and download it.

In [ ]:
model_name = 'question_match_model'

model = Model(ws, name=model_name)
print(model.name, model.version, model.url, sep="\n")

In [ ]:
model.download(target_dir=".", exist_ok=True)

## Create Scoring Script

We use the writefile magic to write the contents of the below cell to `score.py` which includes the  `init` and `run` 
functions required by AML.
- The init() function typically loads the model into a global object.
- The run(input_data) function uses the model to predict a value based on the input_data.

In [ ]:
%%writefile score.py

import pandas as pd
import json
from duplicate_model import DuplicateModel
import logging
import timeit as t

def init():
    logger = logging.getLogger("scoring_script")
    global model
    model_path = "model.pkl"
    questions_path = "./data_folder/questions.tsv"
    start = t.default_timer()
    model = DuplicateModel(model_path, questions_path)
    end = t.default_timer()
    loadTimeMsg = "Model loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)


def run(body):
    logger = logging.getLogger("scoring_script")
    json_load_text = json.loads(body)
    text_to_score = json_load_text["input"]
    start = t.default_timer()
    resp = model.score(text_to_score)
    end = t.default_timer()
    logger.info("Prediction took {0} ms".format(round((end-start)*1000, 2)))
    return json.dumps(resp)
